This script train the graph reasoning algorithm use detected concept for XAI

In [ ]:
!conda activate VRX
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from torch_geometric.data import DataLoader
import torch.nn as nn
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
import sys
import random
import numpy as np
from src.image_folder import make_dataset
from torch_geometric.data import Data
import os
import torch.nn.functional as f
from torch.nn import Parameter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
from torch_geometric.nn import GraphConv
from typing import Union, Tuple
from torch_geometric.typing import OptTensor, OptPairTensor, Adj, Size
import argparse
from torch import Tensor
from torch.nn import Linear
from torch_geometric.nn.conv import MessagePassing
import torch.nn.functional as F

In [ ]:
class VR_graph(InMemoryDataset): # create training dataset
    def __init__(self, root, transform=None, pre_transform=None):
        self.root = root
        super(VR_graph, self).__init__(root, transform, pre_transform)
        self.data, self.slices, self.XAI_act = torch.load(self.processed_file_names[0])

In [ ]:
%%add_to VR_graph
@property
def raw_file_names(self):
    return []

@property
def processed_file_names(self):
    return [os.path.join(self.root, 'vec2graph', 'Xception_XAI_graph_training.dataset')]

def download(self):
        pass
'''useful for utils'''

def read_txt(self, path):
    f = open(path, 'r')
    b = f.read()
    pos_graph = eval(b)
    f.close()
    return pos_graph

def choose_graph(self, list_mix):
    while 1:
        pos_graph_path = random.choice(list_mix)
        if '_graph.' in pos_graph_path:
            return pos_graph_path
'''process for contrastive loss'''

Get graph data list, where graph data is a torch_geometric data reprentation of a graph.\
For each class, generate 2 positive and 4 negative graphs [+1,-1,-1, +1,-1,-1]
graph_data_list has 1*18 graph representations.

In [ ]:
%%add_to VR_graph
def _get_3class_eij(self):
    class_list = ['fire_engine', 'ambulance', 'school_bus']
    graph_data_list = []
    
    # for (label=[0,1,2], cate=[class_list])
    for label, cate in enumerate(class_list): # 3 category, 6 graph for each class [+1,-1,-1, +1,-1,-1]
        
        # pos_class_g is an array of paths of training images of a given class
        pos_class_path = os.path.join('./result/img2vec/', 'train', cate, cate) # real + eij
        pos_class_g = make_dataset(pos_class_path) 
        
        for i in range(2): # each batch we choose 2 random instance for each category
            # positive
            # Choose a random image from pos_class_g
            graph_path = self.choose_graph(pos_class_g)
            graph = self.read_txt(graph_path)
            
            # x is 4*512, where each row is the embedding of a node
            # y = [1,2,3], target to train against
            x = torch.ones((len(graph), 512))  # (4, 512)
            for n, vec in enumerate(graph.values()):
                x[n] = torch.LongTensor(vec)
            y = torch.FloatTensor([label + 1]) # label of each category
            
            # Construct edge (edge_index representation) (fully connected)
            # edge is 2*|E|
            source_nodes = []
            target_nodes = []
            start_choice = [n for n in range(len(graph))] # [0,1,2,3]
            for startpt in start_choice:
                # do not set loop
                end_choice = start_choice.copy()
                end_choice.remove(startpt)
                for endpt in end_choice:
                    source_nodes.append(startpt)
                    target_nodes.append(endpt)
            edge = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

            '''edge_feature'''
            '''1 edge feature extraction'''
            # Each edge has 4 features [xj,yj,xi,yi]
            # e_all is 4*4 adjacency matrix
            #edge_feature is 4*4*4
            edge_path = graph_path.replace('_graph.', '_edge.')
            edge_feature = self.read_txt(edge_path)
            e_all = torch.zeros((len(edge_feature), 4, 4))  # (4, 4, 4)
            for startpt, startpt_values in edge_feature.items():
                for endpt, endpt_values in startpt_values.items():
                    e_all[startpt][endpt] = torch.tensor(endpt_values)
                    
            '''2 get edge_feature form edge_index'''
            # e is an array of edge features (edge attribute)
            # e is |E|*4
            e = []
            for index, source_p in enumerate(source_nodes):
                target_p = target_nodes[index]
                e.append(np.array(e_all[source_p][target_p]))
            e = torch.Tensor(e)
            data = Data(x=x, edge_attr=e, edge_index=edge, y=y)
            graph_data_list.append(data)

            #  negative (for each class do same thing on remaining classes)
            remain_cate = class_list.copy()
            remain_cate.remove(cate)
            for eij_label, neg_cate in enumerate(remain_cate):
                neg_class_path = os.path.join('./result/img2vec/', 'train', neg_cate, cate)
                neg_class_g = make_dataset(neg_class_path)
                neg_graph_path = self.choose_graph(neg_class_g)
                graph = self.read_txt(neg_graph_path)
                x = torch.ones((len(graph), 512))  # (3, 512)
                for n, vec in enumerate(graph.values()):
                    x[n] = torch.LongTensor(vec)
                y = torch.FloatTensor([3 + 2 * (label) + (eij_label+1)])  # label of each category (45,67,89)
                '''edge_index'''
                source_nodes = []
                target_nodes = []
                start_choice = [n for n in range(len(graph))]
                for startpt in start_choice:
                    # do not set loop
                    end_choice = start_choice.copy()
                    end_choice.remove(startpt)
                    for endpt in end_choice:
                        source_nodes.append(startpt)
                        target_nodes.append(endpt)
                edge = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

                '''edge_feature'''
                '''1 edge feature extraction'''
                edge_path = neg_graph_path.replace('_graph.', '_edge.')
                edge_feature = self.read_txt(edge_path)
                e_all = torch.zeros((len(edge_feature), 4, 4))  # (4, 4, 4)
                for startpt, startpt_values in edge_feature.items():
                    for endpt, endpt_values in startpt_values.items():
                        e_all[startpt][endpt] = torch.tensor(endpt_values)
                '''2 get edge_feature form edge_index'''
                e = []
                for index, source_p in enumerate(source_nodes):
                    target_p = target_nodes[index]
                    e.append(np.array(e_all[source_p][target_p]))
                e = torch.Tensor(e)
                data = Data(x=x, edge_attr=e, edge_index=edge, y=y)
                graph_data_list.append(data)

    return graph_data_list

Form as graph and store activation value for given class

In [ ]:
def _get_1class_XAI(self, cate, label):
    class_list = ['fire_engine', 'ambulance', 'school_bus']

    graph_data_list = []
    XAI_activation_list = []

    pos_class_path = os.path.join(self.root, 'img2vec', cate + '_detect_graph')
    for roots, dirs, files in os.walk(pos_class_path):
        for i, file in enumerate(files):  # we only need limited number for each class
            if '_graph' in file: # first process graph
                try:
                    graph_path = os.path.join(roots,file)
                    graph = self.read_txt(graph_path)
                    '''x, y'''
                    x = torch.ones((len(graph), 2048))  # (4, 2048)
                    for n, vec in enumerate(graph.values()):
                        x[n] = torch.tensor(vec)
                    y = torch.FloatTensor([label + 1]) # label of each category
                    '''edge_index'''
                    source_nodes = []
                    target_nodes = []
                    start_choice = [n for n in range(len(graph))]
                    for startpt in start_choice:
                        # do not set loop
                        end_choice = start_choice.copy()
                        end_choice.remove(startpt)
                        for endpt in end_choice:
                            source_nodes.append(startpt)
                            target_nodes.append(endpt)
                    edge = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

                    '''edge_feature'''
                    '''1 edge feature extraction'''
                    edge_path = graph_path.replace('_graph.', '_edge.')
                    edge_feature = self.read_txt(edge_path)
                    e_all = torch.zeros((len(edge_feature), 4, 4))  # (4, 4, 4)
                    for startpt, startpt_values in edge_feature.items():
                        for endpt, endpt_values in startpt_values.items():
                            e_all[startpt][endpt] = torch.tensor(endpt_values)
                    '''2 get edge_feature form edge_index'''
                    e = []
                    for index, source_p in enumerate(source_nodes):
                        target_p = target_nodes[index]
                        e.append(np.array(e_all[source_p][target_p]))
                    e = torch.Tensor(e)
                    '''XAI_activation'''
                    act_path = graph_path.replace('_graph.txt', '_XAI.npy')
                    act_vector = np.load(act_path)
                    # merge the act into y label
                    data_pos = Data(x=x, edge_attr=e, edge_index=edge, y=[(y, cate, torch.FloatTensor(act_vector))]) # y is list
                    # graph_data_list.append(data)

                    #  negative
                    remain_cate = class_list.copy()
                    remain_cate.remove(cate)
                    for eij_label, neg_cate in enumerate(remain_cate): # only load graph, no activation
                        neg_cate_file = cate + '2' + neg_cate # e.g. fire_engine2ambulance
                        neg_graph_path = graph_path.replace(cate, neg_cate_file)
                        graph = self.read_txt(neg_graph_path)
                        x = torch.ones((len(graph), 2048))  # (4, 512)
                        for n, vec in enumerate(graph.values()):
                            # x[n] = torch.LongTensor(vec) # will make all of them 0
                            x[n] = torch.tensor(vec)
                        y = torch.FloatTensor([label + 1])  # label of each category (45,67,89)
                        '''edge_index'''
                        source_nodes = []
                        target_nodes = []
                        start_choice = [n for n in range(len(graph))]
                        for startpt in start_choice:
                            # do not set loop
                            end_choice = start_choice.copy()
                            end_choice.remove(startpt)
                            for endpt in end_choice:
                                source_nodes.append(startpt)
                                target_nodes.append(endpt)
                        edge = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

                        '''edge_feature'''
                        '''1 edge feature extraction'''
                        edge_path = neg_graph_path.replace('_graph.', '_edge.')
                        edge_feature = self.read_txt(edge_path)
                        e_all = torch.zeros((len(edge_feature), 4, 4))  # (4, 4, 4)
                        for startpt, startpt_values in edge_feature.items():
                            for endpt, endpt_values in startpt_values.items():
                                e_all[startpt][endpt] = torch.tensor(endpt_values)
                        '''2 get edge_feature form edge_index'''
                        e = []
                        for index, source_p in enumerate(source_nodes):
                            target_p = target_nodes[index]
                            e.append(np.array(e_all[source_p][target_p]))
                        e = torch.Tensor(e)
                        # '''XAI_activation'''
                        data_neg = Data(x=x, edge_attr=e, edge_index=edge, y=(y, neg_cate))
                        data_pos.y.append(data_neg) # merge into y
                    '''finally add all into data'''
                    graph_data_list.append(data_pos)
                except:
                    print(graph_path)



    return graph_data_list, XAI_activation_list